In [1]:
import time
import pandas as pd
import openai
import csv
import os
import pickle

In [2]:
print('Enter OpenAI API key:')
openai.api_key = input()

os.environ['OPENAI_API_KEY']=openai.api_key

Enter OpenAI API key:


In [4]:
# #open a csv file and read contents into a list
# notes = pd.read_csv('notes.csv', header=None)   
with open('notes.pkl', 'rb') as f:
    data = pickle.load(f)
print(len(data[:5]))

5


In [5]:
#send each line of the df to an OpenAI GPT-3 API call to generate a chat responce
base_prompt = "You are an assistant for a doctor writing the discharge note for a patient to understand. This should include what happened in the visit, what they should do at home, and any medication they need to take upon release. Make sure to specify any medication changes. Assume the patient has a 5th grade level of medical understanding. Here is their clinical history: \n\n"
specificInstructions = "Write out the full words for any medical abbreviations in the clinical history and and use generic medicine names.  Make sure to specify any medication changes"
special_instructions =  "\n\n And here are specific instructions that the doctor has specified: " + specificInstructions

responses = []

for row in data[:5]:
    prompt = base_prompt + row[0] + special_instructions
    response = openai.ChatCompletion.create(
        model= "gpt-4",
        messages= [{"role": "user", "content": prompt}],
    )
    responses.append(response.choices[0].message['content'])
    time.sleep(1)

In [6]:
print(responses)

["Dear [Patient's Name],\n\nDuring your visit to the doctor, we talked about your high blood pressure (also called hypertension) and high cholesterol levels. These conditions can affect your heart and blood vessels, making it difficult for your blood to flow easily. To help you feel better and lower these levels, we discussed the following instructions for you to follow at home and medications you need to take.\n\nAt home, it is important to focus on a healthy lifestyle. You should:\n\n1. Eat a diet that is low in salt, fat, and cholesterol. This means choosing more fruits, vegetables, whole grains, and lean proteins. You should avoid or limit fast food, fried food, and sugary treats.\n\n2. Get regular exercise by being active for at least 30 minutes a day, 5 days a week. You can go for a brisk walk, swim, or bicycle, for example.\n\n3. Maintain a healthy weight. If you are overweight, try to lose weight slowly by eating healthier and exercising more.\n\n4. Limit alcohol to no more tha

In [7]:
#save the responses to a csv file
with open("discharge_instructions.csv", "w") as f:
    write = csv.writer(f)
    write.writerow(responses)

In [40]:
#use chatGPT to evaluate each discharge instruction on the PEMAT-P system
for inst in responses:
    prompt = "You are a research assistant using the PEMAT-P assessment framework to evaluate this discharge note according to the 26 provided metrics in the PEMAT-P framework. Just an overall PEMAT-P score out of 100 \n\n" + inst
    response = openai.ChatCompletion.create(
        model= "gpt-4",
        messages= [{"role": "user", "content": prompt}],
    )
    print(response.choices[0].message.content)
    time.sleep(1)


{
  "content": "Using the PEMAT-P assessment framework, this discharge note scores a 90 out of 100. The note is clear, concise, uses simple language, and covers important details about the patient's diagnosis, treatment, and medications. It also includes relevant instructions for post-hospital care and follow-up appointments. However, there is room for improvement in terms of more detailed instructions on activity restrictions, signs of potential complications to watch out for, and contact information for the healthcare team.",
  "role": "assistant"
}
